In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import src.kgbench as kg


In [12]:
import os

# Get the current directory
current_dir = os.getcwd()

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

# Check if the current directory is already the parent directory
if current_dir != '/Users/macoftraopia/Documents/GitHub/RGCN-Explainer':
    # Set the parent directory as the current directory
    os.chdir(parent_dir)
!pwd

/Users/macoftraopia/Documents/GitHub/RGCN-Explainer


In [13]:
#functions
def keep_columns_with_non_zero_values(df):
    # Get the column names with non-zero values
    non_zero_columns = df.columns[df.astype(bool).any(axis=0)]

    # Create a new DataFrame with only the columns containing non-zero values
    modified_df = df[non_zero_columns]

    return modified_df

from matplotlib import table


def dataframe_to_overleaf_table(df):
    # Begin the table
    table_code = "\\begin{table}[ht]\n"
    table_code += "\\centering\n"
    table_code += "\\caption{Insert caption here}\n"
    table_code += "\\label{tab:my_label}\n"

    # Generate the table header
    table_code += "\\begin{tabular}{ |"

    # Generate the column alignment specification
    for _ in range(df.shape[1] + 1):
        table_code += "c |"

    table_code += "}\n"
    table_code += "\\hline\n"

    # Generate the column headers
    table_code += " & " + " & ".join(df.columns) + " \\\\\n \hline"
    table_code += "\\hline\n"

    # Generate the table rows
    for idx, row in df.iterrows():
        row_values = []
        for val in row:
            if val == 0:
                row_values.append("")
            else:
                row_values.append(str(np.round(val,3)))

        table_code += str(idx) + " & " + " & ".join(row_values) + " \\\\\n \hline"

    # End the table
    table_code += "\\hline\n"
    table_code += "\\end{tabular}\n"
    table_code += "\\end{table}"

    return table_code


def select_columns_with_values_above_threshold(df, threshold):
    # Select columns where there is at least one entry higher than 1
    selected_columns = df.columns[(df > threshold).any()]

    # Create a new DataFrame with the selected columns
    modified_df = df[selected_columns]

    return modified_df

In [14]:
name = 'mdgenre'

In [15]:

if name in ['aifb', 'mutag', 'bgs', 'am', 'mdgenre', 'amplus', 'dmg777k']:
    data = kg.load(name, torch=True, final=False)
if 'IMDb' in name:    
    data = torch.load(f'data/IMDB/finals/{name}.pt')
if 'dbo' in name:
    data = torch.load(f'data/DBO/finals/{name}.pt')

loaded data mdgenre (74.91s).


In [7]:
id = 'one_relation'
sample = 'sample' #or ''
if name == 'aifb':
    df_w = pd.read_csv(f'chk/{name}_chk/Relation_Selection/Important_{id}_{name}_results_{sample}_ones_withheld.csv')
    # df_t = pd.read_csv(f'chk/{name}_chk/Relation_Selection/Important_{id}_{name}_results_{sample}_ones_test.csv')
    # df = pd.concat([df_w,df_t])
    df = df_w
else:
    #df = pd.read_csv(f'chk/{name}_chk/Relation_Selection/Important_{id}_{name}_results_ones_withheld.csv')
    node_labels = data.withheld #([[0, 'LabelA'], [1, 'LabelB'], [2, 'LabelC']])
    df = pd.read_csv(f'chk/{name}_chk/Relation_Selection/Important_{id}_{name}_results_{sample}_ones_withheld.csv')
    df = pd.read_csv(f'chk/{name}_chk/Relation_Selection/Important_{id}_{name}_results_{sample}_ones_withheld.csv')
    label_dict = dict(node_labels)
    df['label'] = df['node_idx'].map(dict(data.withheld))
#df['label'] = df['node_idx'].map(dict(data.withheld))
df.set_index('node_idx', inplace=True)
if name == 'aifb':
    df.drop('type.1', axis=1, inplace=True)
    df.drop('type.2', axis=1, inplace=True)
df = df.fillna(0)
gb = df.groupby('label')   
a = gb.aggregate('mean') #a = gb.sum()
a['group size'] = gb.size()
a = a.fillna(0)
if name == 'IMDb_us':
    a = a[a['group size'] > 10]
    a.drop('type', axis=1, inplace=True)
    a.drop('label.1', axis=1, inplace=True)
if name == 'dbo_gender':
    a.drop('subject.1', axis=1, inplace=True)

a = keep_columns_with_non_zero_values(a)
a_high = select_columns_with_values_above_threshold(a,0.4)

a_high

,fax,homepage,name,phone,photo,publication,type,worksAtProject,group size
label,,,,,,,,,
0,1.0,0.0,1.0,0.0,0.0,0.0,0.75,0.00,4
1,0.0,0.5,0.0,0.0,1.0,0.0,0.00,0.75,4
2,0.0,0.0,0.0,1.0,0.0,1.0,0.00,0.00,4
3,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,4


In [53]:

print(dataframe_to_overleaf_table(a_high))

\begin{table}[ht]
\centering
\caption{Insert caption here}
\label{tab:my_label}
\begin{tabular}{ |c |c |c |c |c |c |c |c |c |c |c |c |c |c |}
\hline
 & birthPlace & careerStation & position & team & wikiPageExternalLink & birthPlace.1 & position.1 & team.1 & subject & hypernym & wasDerivedFrom & isPrimaryTopicOf & group size \\
 \hline\hline
0 & 0.848 & 0.514 & 0.536 & 0.536 &  & 0.848 & 0.536 & 0.536 &  & 0.841 & 0.993 & 0.855 & 138.0 \\
 \hline1 &  &  &  &  & 0.531 &  &  &  & 1.0 &  &  &  & 113.0 \\
 \hline\hline
\end{tabular}
\end{table}


wrong if 

In [18]:
id = 'wrong_if'
if name == 'aifb':
    df_w = pd.read_csv(f'chk/{name}_chk/Relation_Selection/Important_{id}_{name}_results_{sample}_ones_withheld.csv')
    # df_t = pd.read_csv(f'chk/{name}_chk/Relation_Selection/Important_{id}_{name}_results_ones_test.csv')
    # df = pd.concat([df_w,df_t])
    df = df_w
else:
    #df = pd.read_csv(f'chk/{name}_chk/Relation_Selection/Important_{id}_{name}_results_ones_withheld.csv')
    #node_labels = data.withheld #([[0, 'LabelA'], [1, 'LabelB'], [2, 'LabelC']])
    df = pd.read_csv(f'chk/{name}_chk/Relation_Selection/Important_{id}_{name}_results_{sample}_ones_withheld.csv')
    #df = pd.read_csv(f'chk/{name}_chk/Relation_Selection/Important_{id}_{name}_results_{sample}_ones_withheld.csv')
    #label_dict = dict(node_labels)
    #df['label'] = df['node_idx'].map(dict(data.withheld))


df.set_index('node_idx', inplace=True)
if name == 'aifb':
    df.drop('type.1', axis=1, inplace=True)
    df.drop('type.2', axis=1, inplace=True)
df = df.fillna(0)
gb = df.groupby('label')   
a = gb.aggregate('mean') #a = gb.sum()
a['group size'] = gb.size()
a = a.fillna(0)
if name == 'IMDb_us':
    a = a[a['group size'] > 10]
    a.drop('label.1', axis=1, inplace=True)
    a.drop('type', axis=1, inplace=True)
if name == 'dbo_gender':
    a.drop('subject.1', axis=1, inplace=True)
a = keep_columns_with_non_zero_values(a)
a_high = select_columns_with_values_above_threshold(a,0.4)
a_high

,P10,P102,P103,P1038,P1040,P1050,P106,P1066,P108,P109,...,P921,P935,P937,P941,P945,P97,P973,P989,P990,group size
label,,,,,,,,,,,,,,,,,,,,,
0,1.00,1.00,1.00,1.00,0.75,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,4
1,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,...,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,4
2,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,...,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,4
3,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,4
4,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,...,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,4
5,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,4
6,0.50,0.25,0.25,0.25,0.50,0.25,0.75,0.25,0.25,0.25,...,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,4
7,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,...,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,4
8,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,4


In [19]:
print(dataframe_to_overleaf_table(a_high))

\begin{table}[ht]
\centering
\caption{Insert caption here}
\label{tab:my_label}
\begin{tabular}{ |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |}
\hline
 & P10 & P102 & P103 & P1038 & P1040 & P1050 & P106 & P1066 & P108 & P109 & P118 & P119 & P1191 & P1196 & P1283 & P1303 & P1317 & P1327 & P1340 & P1343 & P1344 & P135 & P138 & P140 & P1411 & P1412 & P1424 & P1431 & P144 & P1442 & P1449 & P1455 & P1476 & P1477 & P1532 & P154 & P1559 & P1576 & P161 & P162 & P166 & P170 & P172 & P175 & P179 & P18 & P180 & P1801 & P1814 & P1830 & P185 & P1853 & P1875 & P1877 & P1884 & P1889 &

In [22]:
import pandas as pd

def select_columns_with_values_above_threshold(df, threshold):
    # Select columns where there is at least one entry higher than 1
    selected_columns = df.columns[(df > threshold).any()]

    # Create a new DataFrame with the selected columns
    modified_df = df[selected_columns]

    return modified_df


a_high = select_columns_with_values_above_threshold(a,0.4)
a_high

,name,publication,worksAtProject,group size
label,,,,
0,0.642857,0.071429,0.071429,14
1,0.000000,0.250000,0.500000,4
2,0.000000,1.000000,0.000000,11
3,0.166667,0.166667,0.333333,6


In [23]:
print(dataframe_to_overleaf_table(a_high))

\begin{table}[ht]
\centering
\caption{Insert caption here}
\label{tab:my_label}
\begin{tabular}{ |c |c |c |c |c |}
\hline
 & name & publication & worksAtProject & group size \\
 \hline\hline
0 & 0.643 & 0.071 & 0.071 & 14.0 \\
 \hline1 &  & 0.25 & 0.5 & 4.0 \\
 \hline2 &  & 1.0 &  & 11.0 \\
 \hline3 & 0.167 & 0.167 & 0.333 & 6.0 \\
 \hline\hline
\end{tabular}
\end{table}


test set backward

IMDb us dataset (extracted from Triply IMDb dataset selecting the subset of movies whose country is the US):

In [86]:
from RGCN_stuff.src.rgcn_explainer_utils import get_relations


name = 'IMDb_us'
data = torch.load(f'data/IMDB/finals/{name}.pt')
#label is 5
for i in data.triples:
    if i[1] == 5:
        print(data.i2e[i[0]], data.i2rel[i[1]], data.i2e[i[2]])


In [102]:
id = 'wrong_if'
id_test = 'withheld'
name = 'IMDb_us'
df = pd.read_csv(f'chk/{name}_chk/Important_{id}_{name}_results_ones_{id_test}.csv')
df.set_index('node_idx', inplace=True)
df = df.fillna(0)
gb = df.groupby('label')   
#a = gb.aggregate('mean') 
a = gb.sum()
a['group size'] = gb.size()
#a = a.fillna(0)
a = keep_columns_with_non_zero_values(a)
a_high = select_columns_with_values_above_threshold(a,2)
a_high = keep_columns_with_non_zero_values(a_high)
a_high
print(dataframe_to_overleaf_table(a_high))

\begin{table}[ht]
\centering
\caption{Insert caption here}
\label{tab:my_label}
\begin{tabular}{ |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |c |}
\hline
 & hasPhotoCollection & date & title & type & SeeAlso & label.1 & subClassOf & sameAs & subject & based_near & page & actor & actor_actorid & actor_name & actor_netflix_id & actor_nytimes_id & cinematographer & cinematographer_cinematographerid & cinematographer_name & country & country_areaInSqKm & country_capital & country_continent & country_currency & country_fips_code & country_id & country_iso_alpha2 & country_iso_alpha3 & country_iso_numeric & country_languages & country_name & country_population & director & dubbing_performance & editor & editor_editorid & editor_name & executive_producer & featured_film_location & film